### Computer Graphics: illumination models and surfaces
###### by Hamed Shah-hosseini
Explanation at: https://www.pinterest.com/HamedShahHosseini/
<br>https://github.com/ostad-ai/Computer-Graphics

In [1]:
from matplotlib import pyplot as plt
import numpy as np
from math import sin,cos,pi,tan
from ipywidgets import interact

In [3]:
# for github
# light source and flat shading,with hidden surface removal
#perspective projection with pygame, is good for demo
#point source position
lightPos=np.array([0.,0.,100.])
# point source color
lightCol=[1.,1.,1.]
# ambient light color
lightAmb=[1.,1.,1.]
#surface parameters for illumination
ka,kd,ks,ns=.25,.5,.9,50
#parameters for the projection
width,height=640,480
aspect=width/height
near,far=1.,100.
fovy=30.
#vertices of a cube
vertex_list=np.array([[-1.,-1.,-1.],
                     [-1.,1.,-1.],
                      [1.,1.,-1.],
                      [1.,-1.,-1.],
                      [-1.,-1.,1.],
                      [-1.,1.,1.],
                      [1.,1.,1.],
                      [1.,-1.,1.]
                     ])
vertex_list[:,2]+=-12 #decrease z values
vertex_list[:,0]+=2. #increase x values

#faces of the cube
face_list=np.array([[4,7,6,5],[7,3,2,6],
                    [3,0,1,2],[4,5,1,0],
                   [5,6,2,1],[4,0,3,7]])

# colors of the faces
RED=(255,0,0); GREEN=(0,255,0); BLUE=(0,0,255)
CYAN=(0,255,255); YELLOW=(255,255,0); PURPLE=(255,0,255) 
face_colors=[RED,GREEN,BLUE,CYAN,YELLOW,PURPLE]

# creating another cube
face_list2=face_list.copy() 
vertex_list2=vertex_list.copy()
vertex_list2[:,0]+=-4.

#tco cubes as meshes, one with no specular refl.
meshes=[(vertex_list,face_list,False),(vertex_list2,face_list2,True)]

def normalize(vector):
    norm=np.linalg.norm(vector)
    if norm>0:
        return vector/norm
    else:
        return vector
    
def compute_normals(f_list,vec3s):
    norms=[]
    for face in f_list:
        mx,my,mz=0.,0.,0.
        N=len(face)
        verts=np.zeros((N,3))
        for i in range(N):
            verts[i]=vec3s[face[i]].copy()
        for i in range(N):
            mx+=(verts[i,1]-verts[(i+1)%N,1])*\
                (verts[i,2]+verts[(i+1)%N,2])
            my+=(verts[i,2]-verts[(i+1)%N,2])*\
                (verts[i,0]+verts[(i+1)%N,0])
            mz+=(verts[i,0]-verts[(i+1)%N,0])*\
                (verts[i,1]+verts[(i+1)%N,1])
        norms.append(np.array([mx,my,mz]))
    return np.asarray(norms)

def to_homogeneous(x,y,z):
    return np.array([x,y,z,np.ones_like(x)])

#fovY is in degrees
def perspective(fovY,aspect,zNear,zFar):
    epsilon=.01; fovY*=pi/360.
    if fovY<epsilon: fovY=epsilon
    f=1./tan(fovY)
    return np.array([[f/aspect,0.,0.,0.],
                     [0.,f,0.,0.],
                     [0.,0.,(zNear+zFar)/(zNear-zFar),
                      2.*zNear*zFar/(zNear-zFar)],
                     [0.,0.,-1.,0.]
                    ])
# matrix for rotation around y
def matrix_rotY(teta):
    c=cos(teta); s=sin(teta)
    return np.array([[c,0.,s,0.],
                     [0,1.,0.,0.],
                     [-s,0.,c,0.],
                     [0.,0.,0.,1.]])        

# matrix for rotation around x
def matrix_rotX(teta):
    c=cos(teta); s=sin(teta)
    return np.array([[1.,0.,0.,0.],
                     [0,c,-s,0.],
                     [0.,s,c,0.],
                     [0.,0.,0.,1.]])        

# matrix for rotation around z
def matrix_rotZ(teta):
    c=cos(teta); s=sin(teta)
    return np.array([[c,-s,0.,0.],
                     [s,c,0.,0.],
                     [0.,0.,1.,0.],
                     [0.,0.,0.,1.]])        

# rotation around axis z             
def rotateXYZ(v_list,tx,ty,tz,center=True):
    matXYZ=np.dot(matrix_rotX(tx),matrix_rotY(ty))
    matXYZ=np.dot(matXYZ,matrix_rotZ(tz))
    if center:
        xyz_mean=v_list.mean(axis=0)
        return xyz_mean+matrix_vector(matXYZ,
                                v_list-xyz_mean)
    else:
        return matrix_vector(matXYZ,vertex_list)
# project 3D vector into new space with the matrix    
def matrix_vector(matrix,vec3):
    temp=np.dot(matrix,to_homogeneous(vec3[:,0],vec3[:,1],vec3[:,2]))
    xs,ys,zs=temp[0]/temp[3],temp[1]/temp[3],temp[2]/temp[3]
    return np.vstack([xs,ys,zs]).T   

#compute perspective matrix
matrix_pers=perspective(fovy,aspect,near,far)

def run(tx,ty,tz):
    eye=np.array([0.,0.,0.])
    for (v_list,f_list,specFlag) in meshes:
        xyz=rotateXYZ(v_list,tx,ty,tz)
        normals=compute_normals(f_list,xyz)
        xyz2=matrix_vector(matrix_pers,xyz)
        #scaling into width and height 
        x_vp,y_vp=.5*width*(xyz2[:,0]+1),.5*height*(xyz2[:,1]+1)
        for id,face in enumerate(f_list):
            coords=[]
            eye2look=xyz[face[0]]-eye
            if np.dot(eye2look,normals[id])>=0: continue
            N=normalize(normals[id])
            # for reducing computation,only for mean point of face, illumination is computed
            xyz_f_mean=xyz[face].mean(axis=0)
            L=normalize(lightPos-xyz_f_mean)
            V=normalize(eye-xyz_f_mean)
            H=normalize(L+V)
            NL=np.dot(N,L)
            NH=np.dot(N,H)
            iTotal=ka*np.array(lightAmb)
            if NL>0 :
                iTotal+=kd*np.array(lightCol)*NL
                if NH>0 and specFlag:
                    iTotal+=ks*np.array(lightCol)*(NH**ns)
            fCol=iTotal*np.array(face_colors[id%len(face_colors)])
            fCol=np.clip(fCol,0,255)/255.
            for i in range(len(face)):
                coords.append([x_vp[face[i]],y_vp[face[i]]])
            p=plt.Polygon(coords,facecolor=list(fCol))
            plt.gca().add_patch(p)
    plt.xlim([0,width]); plt.ylim([0,height])
    plt.axis('off'); plt.show()
    
interact(run,tx=(-3.1,3.14,.1),ty=(-3.1,3.14,.1),tz=(-3.1,3.14,.1))

interactive(children=(FloatSlider(value=0.0, description='tx', max=3.14, min=-3.1), FloatSlider(value=0.0, des…

<function __main__.run(tx, ty, tz)>